In [42]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer



In [43]:
# Carga del dataset procesado
df = pd.read_csv('..\\Data\processed\data_processed.csv', sep=',')

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Lenovo G40 80\AppData\Local\Temp\ipykernel_4332\808941136.py:2: SyntaxWarning: invalid escape sequence '\p'
  df = pd.read_csv('..\\Data\processed\data_processed.csv', sep=',')


In [44]:

# Revisión inicial de los datos
print(df.head())
print(df.info())
print(df.describe())

   particion periodo  nota_final_materia  points_possible      score  \
0          0  1-2022                 9.0        99.593262  88.333333   
1          1  1-2022                 9.0        99.593262  88.333333   
2          2  1-2022                 9.0        99.593262  88.333333   
3          3  1-2022                 9.0        99.593262  88.333333   
4          4  1-2022                 9.0        99.593262  88.333333   

                              user_uuid                           course_uuid  \
0  13df535e-065c-4593-98ea-5b1e29015b7d  09614210-fce2-48bc-93e3-bc4bd441fe00   
1  13df535e-065c-4593-98ea-5b1e29015b7d  09614210-fce2-48bc-93e3-bc4bd441fe00   
2  13df535e-065c-4593-98ea-5b1e29015b7d  09614210-fce2-48bc-93e3-bc4bd441fe00   
3  13df535e-065c-4593-98ea-5b1e29015b7d  09614210-fce2-48bc-93e3-bc4bd441fe00   
4  13df535e-065c-4593-98ea-5b1e29015b7d  09614210-fce2-48bc-93e3-bc4bd441fe00   

                         course_name     legajo  
0  Progressive homogeneous str

# Análisis de Características

In [45]:
# Verificando valores únicos en columnas categóricas
print(df['course_name'].unique())
print(df['periodo'].unique())

# Verificando estadísticos de características numéricas
print(df['nota_final_materia'].describe())
print(df['score'].describe())

['Progressive homogeneous structure' 'Switchable radical service-desk'
 'Re-contextualized hybrid initiative'
 'Face-to-face incremental instruction set' 'Customizable hybrid forecast'
 'Virtual intermediate Graphical User Interface'
 'Business-focused bandwidth-monitored matrices'
 'Self-enabling discrete info-mediaries' 'Total radical adapter'
 'Future-proofed zero tolerance ability'
 'Fully-configurable contextually-based Graphical User Interface'
 'Stand-alone next generation infrastructure'
 'Quality-focused 24/7 structure' 'Re-engineered tangible system engine'
 'Persevering content-based neural-net'
 'Implemented 24/7 Graphical User Interface' 'Operative local database'
 'Proactive tangible paradigm' 'Multi-channeled user-facing architecture'
 'User-friendly systemic groupware'
 'Customizable fault-tolerant implementation'
 'Centralized asymmetric projection' 'Pre-emptive real-time collaboration'
 'Profound web-enabled algorithm' 'Cross-platform bottom-line approach'
 'Digitized

El análisis muestra una amplia variedad de nombres de cursos y períodos, con las notas finales y los puntajes de los alumnos distribuidos en rangos variados.

In [48]:
# 1. Codificación de variables categóricas
def preprocess_categorical(df):
    # Codificación de 'course_name' con One-Hot Encoding
    df_encoded = pd.get_dummies(df, columns=['course_name'], drop_first=True)
    return df_encoded

# 2. Creación de nuevas características
def create_features(df):
    # Extraer año y mes del periodo
    df['periodo_year'] = df['periodo'].str.split('-').str[0].astype(int)
    df['periodo_month'] = df['periodo'].str.split('-').str[1].astype(int)
    
    # Relación entre score y puntos posibles
    df['score_to_points_ratio'] = df['score'] / df['points_possible']
    
    return df

# 3. Normalización de datos
def normalize_data(df):
    scaler = StandardScaler()
    numerical_features = ['nota_final_materia', 'points_possible', 'score', 'score_to_points_ratio']
    
    # Normalización de características numéricas
    df[numerical_features] = scaler.fit_transform(df[numerical_features])
    return df

# Aplicar las funciones
df = preprocess_categorical(df)
df = create_features(df)
df = normalize_data(df)

# Resultados
print(df.head())

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [46]:
# Columnas categóricas y numéricas
categorical_features = ['course_name', 'legajo']  # Columnas categóricas en el DataFrame
numerical_features = ['nota_final_materia', 'points_possible', 'score']  # Columnas numéricas en el DataFrame

# Transformador para características categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación de valores faltantes
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))    # Codificación one-hot
])

# Transformador para características numéricas
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Imputar valores faltantes
    ('scaler', StandardScaler())                   # Normalización
])

# Combinador de transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Ajuste y transformación de datos
X = preprocessor.fit_transform(df)

# Verifica la forma de X
print("Shape of X:", X.shape)

# Obteniendo los nombres de las características después de la transformación
num_features = numerical_features
cat_feature_names = list(preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out())
feature_names = num_features + cat_feature_names

print("Feature names:", feature_names)

# Convertiendo el resultado a DataFrame
X_df = pd.DataFrame(X, columns=feature_names)

print(X_df.head())


Shape of X: (184156, 1165)
Feature names: ['nota_final_materia', 'points_possible', 'score', 'x0_Adaptive 6thgeneration Graphic Interface', 'x0_Adaptive analyzing encoding', 'x0_Adaptive cohesive definition', 'x0_Adaptive foreground info-mediaries', 'x0_Adaptive heuristic framework', 'x0_Adaptive logistical toolset', 'x0_Adaptive national collaboration', 'x0_Adaptive optimizing support', 'x0_Adaptive transitional adapter', 'x0_Advanced bandwidth-monitored software', 'x0_Advanced didactic installation', 'x0_Ameliorated asymmetric intranet', 'x0_Ameliorated didactic customer loyalty', 'x0_Ameliorated grid-enabled ability', 'x0_Ameliorated homogeneous system engine', 'x0_Ameliorated motivating ability', 'x0_Ameliorated regional instruction set', 'x0_Ameliorated uniform contingency', 'x0_Assimilated analyzing complexity', 'x0_Assimilated asymmetric flexibility', 'x0_Assimilated cohesive contingency', 'x0_Assimilated exuding groupware', 'x0_Assimilated system-worthy forecast', 'x0_Automated

Explicación de Cambios
Columnas Categóricas y Numéricas: Se ajustaron para que coincidan con las columnas (course_name y legajo para categóricas; nota_final_materia, points_possible, y score para numéricas).

Eliminación del Transformador de Texto: Como no tengo columnas de texto en el DataFrame, se eliminó el transformador de texto.

Creación del DataFrame Resultante: Después de la transformación, los nombres de las características se generan a partir de las columnas numéricas y las características resultantes de la codificación one-hot.